# Oral scan bite adjustment

Bite adjustment supports two modes:

1. Adjustment based on intraoral photos: In this mode, input should include upper and lower jaw three-dimensional grids and a set of intraoral photos (at least including frontal occlusion images; if there are left and right occlusion images, precision will be improved).
2. Adjustment based solely on the three-dimensional grid: In this mode, input should include upper and lower jaw three-dimensional grids, and the input dictionary should not contain intraoral photos.

In [ ]:
import os
import glob
import base64
import time
import requests
import json
import trimesh
import urllib
import numpy as np

# Define call rules

Please modify the following code blocks based on the credentials you obtained from us.

In [ ]:
# Chohotech service request URL, sent with the API documentation.
base_url = "<service request URL>"

# Chohotech file service URL, sent with the API documentation.
file_server_url = "<service file server URL>"

# The authentication header must be passed in. Please keep the TOKEN confidential!!! If it is leaked, please contact us immediately to reset it. All tasks using this TOKEN will be charged to your account.
zh_token = "<your company's service Token, sent with the contact>" # All API calls must be authenticated with the token.

user_group = "APIClient" # User group, usually named APIClient.

# Your company's user_id, sent with the API documentation.
user_id = "<your company's user_id>"

# If you have received creds.json, it will be read directly below.
if os.path.exists('../../creds.json'):
    creds = json.load(open('../../creds.json', 'r'))
    base_url = creds['base_url']
    file_server_url = creds['file_server_url']
    zh_token = creds['zh_token']
    user_id = creds['user_id']
    print("loaded creds from creds.json")

In [ ]:
json_call = {
  "spec_group": "mesh-processing",
  "spec_name": "bite-adjustment",
  "spec_version": "1.0-snapshot",
  "user_group": user_group,
  "user_id": user_id
}

### Define Callback URL

Callback will send a POST request to the specified URL after the workflow is completed. The request content is a JSON object with 4 fields:

1. workflow_id (str): the corresponding workflow_id
2. metadata (dict): the metadata you passed when starting the workflow
3. success (bool): whether the workflow succeeded (true for success)
4. reason (str or null): If success is true, this item will be null. Otherwise, it will be a string representing the reason for failure.

If you need a callback, please uncomment the following code block.

In [ ]:
# json_call['metadata'] = { # (optional) metadata can be added here, which will be attached to the callback information. Each item in the dictionary is limited to 128 characters
#     "case_id": "CH-123",
#     "case_name": "ABCDE"
# }
# json_call['notification'] =[ # (optional) callback URL can be added here
#     {"url": "https://www.baidu.com"} # multiple callback URLs can be added here, each in the format {"url": "xxx"}
# ]

## Define Input/Output Blocks

The input block is defined by `input_data` in the JSON file. For 3D meshes, we support the following **input** file formats:

```
"obj"
"stl"
"off"
"ply"
"glb"
"zip": There must be only one mesh file inside the zip, and its extension must be obj, stl, off, ply, or glb.
"tar.gz": There must be only one mesh file inside the tar or tar.gz, and its extension must be obj, stl, off, ply, or glb.
"drc"
```

and the following **output** file formats:
```
"obj"
"stl"
"off"
"ply"
"glb"
"drc"
```


We **strongly recommend** using the `drc` file format for input/output to reduce network transmission time. The generation and reading of drc format files can be done using Google's Draco library: https://github.com/google/draco

The table below shows the file size of the same 3D mesh in different file formats (point precision: 0.001mm):

| File Format | File Size |
| --- | --- |
| stl | 16.7M |
| obj | 13.5M |
| off | 14.2M |
| ply | 7.0M |
| glb | 6.0M |
| drc | 517.3K |


For 3D meshes, input can be passed in two ways:

1. Upload the 3D mesh file to our file service system first, then pass the file pointer to the request body and call the API.
2. Directly pass the base64-encoded binary data in the request body.

For 3D meshes, output can be specified in two ways:

1. Return the file pointer as part of the API response, and users can download the specific binary file from our file system.
2. Directly return the binary data in the form of base64 encoding in the API response.


We **strongly recommend** using the first method for the following reasons:

1. Base64 encoding will increase the bandwidth and increase network latency.
2. To ensure API performance, Choho may reject API requests that are too large. Therefore, for large files, the second method will fail (HTTP CODE 413).

### Upload in file mode

In [ ]:
def upload_file(file_name):
    ext = file_name.split('.')[-1]
    data = open('../../data/' + file_name, 'rb').read()
    resp = requests.get(file_server_url + f"/scratch/{user_group}/{user_id}/upload_url?" +
                        f"postfix={ext}", # Must specify postfix, i.e., the file extension
                        headers={"X-ZH-TOKEN": zh_token}) # Get the signed upload URL
    resp.raise_for_status()

    upload_url = resp.text[1:-1] # Returns a single string JSON "string", can also use json.loads(resp.text)

    resp = requests.put(upload_url, data)  # No auth header is needed for uploading to the cloud storage service

    resp.raise_for_status()
    path = "/".join(urllib.parse.urlparse(upload_url).path.lstrip("/").split("/")[3:])
    urn = f"urn:zhfile:o:s:{user_group}:{user_id}:{path}"
    return urn

json_call["input_data"] = {
    "lower_mesh": {"type":"ply", "data": upload_file('lower_jaw_scan.ply')},
    "upper_mesh": {"type":"drc", "data": upload_file('upper_jaw_scan.drc')},
    "inner_images": {"front": upload_file('inner_front.jpg'), "right": upload_file("inner_right.jpg"),
                     "left": upload_file("inner_left.jpg")}
}

# Submit request

Submit the request using the `/run` POST method.

In [ ]:
headers = {
  "Content-Type": "application/json",
  "X-ZH-TOKEN": zh_token
}

url = base_url + '/run'

response = requests.request("POST", url, headers=headers, data=json.dumps(json_call))
print(response.text)
response.raise_for_status()
create_result = response.json()
run_id = create_result['run_id']

print(run_id)

# Wait for the request to complete

Use the polling API to wait for the request to complete, the API URL is `/run/{run_id}`.

**It is strongly recommended to use the callback method.** Refer to [Define Callback URL](#Define-Callback-URL), the callback method will send a callback message to the given URL at the end of the workflow, regardless of whether the workflow is successful or not, so you do not need to poll the result.

In [ ]:
# Step 2: Polling the status
url = base_url + f"/run/{run_id}"

start_time = time.time()
while time.time()-start_time < 180: # Maximum wait time: 3 minutes
    time.sleep(0.3) # Polling interval
    response = requests.request("GET", url, headers=headers)
    result = response.json()
    if result['completed'] or result['failed']:
        break

if not result['completed']:
    if result['failed']:
        raise ValueError("API error, reason: " + str(result['reason_public']))
    raise TimeoutError("API timeout")

print("API execution time: {}s".format(time.time()-start_time))

# Get the request result

Use `/data/{run_id}` to get all output data of this workflow, use `/data/{run_id}/{key}` to get a specific item of data.

In [ ]:
url = base_url + f"/data/{run_id}"
response = requests.request("GET", url, headers=headers)
result = response.json()

# Visualization

In [ ]:
mesh_l = trimesh.load("../../data/lower_jaw_scan.ply")
mesh_u_data = trimesh.exchange.ply.load_draco(open("../../data/upper_jaw_scan.drc", 'rb'))
mesh_u = trimesh.Trimesh(faces = mesh_u_data['faces'], vertices=mesh_u_data['vertices'])

In [ ]:
print('Before bite adjustment：')
(mesh_l + mesh_u).show()

In [ ]:
print('After bite adjustment:')
(mesh_l.copy().apply_transform(result['lower_align_matrix']).apply_transform(result['lower_bite_adj_matrix']) +
 mesh_u.copy().apply_transform(result['upper_align_matrix']).apply_transform(result['upper_bite_adj_matrix'])).show()

# Bite adjustment without intraloral images

In [ ]:
# delete inner_images from input_data
del json_call['input_data']['inner_images']

In [ ]:
url = base_url + '/run'

response = requests.request("POST", url, headers=headers, data=json.dumps(json_call))
response.raise_for_status()
create_result = response.json()
run_id = create_result['run_id']

print(run_id)

url = base_url + f"/run/{run_id}"

start_time = time.time()
while time.time()-start_time < 180:
    time.sleep(3)
    response = requests.request("GET", url, headers=headers)
    result = response.json()
    if result['completed'] or result['failed']:
        break

if not result['completed']:
    if result['failed']:
        raise ValueError("API error, reason: " + str(result['reason_public']))
    raise TimeoutError("API timeout")

print("API finished in: {}s".format(time.time()-start_time))

url = base_url + f"/data/{run_id}"
response = requests.request("GET", url, headers=headers)
result = response.json()

In [ ]:
print('After bite adjustment')
(mesh_l.copy().apply_transform(result['lower_align_matrix']).apply_transform(result['lower_bite_adj_matrix']) +
 mesh_u.copy().apply_transform(result['upper_align_matrix']).apply_transform(result['upper_bite_adj_matrix'])).show()

If you want to keep upper jaw at exact input coordinate and transform only the lower jaw. The lower jaw transformation matrix can be calculated as follows:

In [ ]:
lmat = np.linalg.solve(np.array(result['upper_bite_adj_matrix']) @ np.array(result['upper_align_matrix']),
                       np.array(result['lower_bite_adj_matrix']) @ np.array(result['lower_align_matrix']))

In [ ]:
(mesh_l.copy().apply_transform(lmat) + mesh_u).show()